# $U(1)$ Gauge Model using L2HMC in graph mode

--------------------------------------------------------------------
### TODO:
* [ ] Look at performance on Cooley (longer training runs).
* [ ] Fit observables to Eq. \ref{eq:therm_time} to determine the thermalization time $\tau$.

\begin{equation} 
f(t) \equiv A \exp^{-t / \tau}+ \,\, B
\label{eq:therm_time}
\end{equation}
* [ ] Look at defining a distance metric as the difference in topological charge between two samples and see what effect adding this as an additional term to the loss function has on the models' ability to tunnel between topological sectors.

--------------------------------------------------------------------

In [1]:
import os
import sys
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.special import i0, i1
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.data_utils import (
    calc_avg_vals_errors, block_resampling, jackknife_err
)

%autoreload 2
%matplotlib notebook

#from l2hmc_eager import gauge_dynamics_eager as gde

In [2]:
#tf.enable_eager_execution()
tfe = tf.contrib.eager

#from u1_model_eager import GaugeModelEager, train_one_iter
from gauge_model import GaugeModel

import utils.gauge_model_helpers as helpers

In [3]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    result /= result[result.argmax()]
    return result[result.size//2:]

## Define parameters

In [18]:
del model

In [23]:
params = {
    'time_size': 8,
    'space_size': 8,
    'link_type': 'U1',
    'dim': 2,
    'beta': 1.,
    'beta_init': 1.,
    'beta_final': 8.,
    'num_samples': 5,
    'num_steps': 10,
    'eps': 0.1,
    'loss_scale': 0.1,
    'loss_eps': 1e-4,
    'learning_rate_init': 1e-4,
    'learning_rate_decay_steps': 1000,
    'learning_rate_decay_rate': 0.98,
    'train_steps': 10000,
    'save_steps': 1000,
    'logging_steps': 50,
    'annealing_steps': 100,
    'annealing_factor': 0.97,
    'clip_value': 10.,
    'rand': False,
    'metric': 'l2',
    'training_samples_steps': 500,
    'training_samples_length': 100
}

## Create model

In [24]:
config = tf.ConfigProto()
tf.reset_default_graph()

In [25]:
model = GaugeModel(params=params, 
                   config=config,
                   sess=None,
                   conv_net=True,
                   hmc=False,
                   log_dir=None,
                   restore=False,
                   eps_trainable=True,
                   aux=True,
                   annealing=True,
                   clip_grads=False)

Creating directory for new run: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_65/
Saving params to: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_65/run_info/parameters.pkl.
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_65/samples_history.
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_65/samples_history/training.
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_65/train_samples.
time_size: 8
space_size: 8
link_type: U1
dim: 2
beta: 1.0
beta_init: 1.0
beta_final: 8.0
num_samples: 5
num_steps: 10
eps: 0.1
loss_scale: 0.1
loss_eps: 0.0001
learning_rate_init: 0.0001
learning_rate_decay_steps: 1000
learning_rate_decay_rate: 0.98
train_steps: 10000
save_steps: 1000
logging_steps: 50
annealing_steps: 100
annealing_factor: 0.97
clip_value: 10.0
rand: False
metric: l2
training_samples_steps: 500
training_samples_length: 100
conv_net: True
hmc: False
aux: True
eps_trainable: True
clip_grads: False
annealing: True
---------------------

In [ ]:
model.train(model.train_steps, kill_sess=False)

----------------------------------------------------------------------------------------------------
     STEP           LOSS       NORM. TIME     ACCEPT %        EPS           BETA           LR      
----------------------------------------------------------------------------------------------------
     0/10000       -8325         0.304        0.0909          0.1            1          0.0001    


In [ ]:
run_steps = [100, 200, 500, 1000, 5000]#, 10000]

In [ ]:
model.beta_final

In [ ]:
for steps in run_steps:
    _ = model.run(steps)

In [39]:
import pickle

with open(model.files['parameters_pkl_file'], 'wb') as f:
    pickle.dump(model.params, f)

In [ ]:
model.sess.graph.collections

In [ ]:
model.sess.graph.get_collection

In [ ]:
model.dynamics.position_fn.summary()

In [ ]:
# Iterate over samples history and calculate observables for each sample.
# `lattice.calc_plaq_observables(samples)` calculates observables for each of
# the samples in the mini-batch.
actions_history = []
avg_plaquettes_history = []
top_charges_history = []
for idx, samples in enumerate(samples_history):
    t0 = time.time()
    observables = np.array(model.lattice.calc_plaq_observables(samples))
    actions, plaqs, charges = observables
    
    actions_history.append(actions)
    avg_plaquettes_history.append(plaqs)
    top_charges_history.append(charges)
    
    print(f'step: {idx}  '
          f'time / step: {time.time() - t0:^6.4g}  '
          f'avg action: {np.mean(actions):^6.4g}  '
          f'avg plaquette: {np.mean(plaqs):^6.4g} '
          f'top charge: {np.mean(charges):^6.4g}')

In [ ]:
_ = helpers.plot_run_data(model.data, 
                          model.params, 
                          model.steps_arr, 
                          model.figs_dir, 
                          skip_steps=1)

In [ ]:
#tf.reset_default_graph()
#model = GaugeModel(params=params,
#                   config=None,
#                   sess=None,
#                   conv_net=False,
#                   hmc=False,
#                   log_dir='../../gauge_logs_graph/run_25',
#                   restore=True)

In [ ]:
samples = np.random.randn(*model.samples.shape)
samples_history = []

In [ ]:
for i in range(500):
    t0 = time.time()
    samples = model.sess.run(model.x_out, feed_dict={model.x: samples})
    samples_history.append(samples)
    print(f'step: {i:^6.4g} time/step: {time.time() - t0:^6.4g}')

In [ ]:
samples_history_conv = np.array(samples_history_conv)
print(samples_history_conv.shape)

In [ ]:
import pickle
samples_history_file = os.path.join(model.info_dir, 'samples_history.pkl')
with open(samples_history_file, 'wb') as f:
    pickle.dump(samples_history_conv, f)